In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
import torch.nn as nn
import pandas as pd
import os
import glob
import sys
#import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

In [ ]:
import random
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [ ]:
data =  pd.read_excel(r"C:\Users\Lenovo\Desktop\data_process\dataset.xlsx")
data_features = data.iloc[:,:10]
data_label = data.iloc[:,-1]

In [ ]:
feature_train,feature_test,label_train,label_test= train_test_split(data_features,data_label,test_size=0.3,random_state = 100,shuffle = True)
feature_train = np.array(feature_train)
feature_test = np.array(feature_test)
label_train = np.array(label_train)
label_test = np.array(label_test)

In [ ]:
class ExcelDataset(Dataset):
    def __init__(self):
        self.feature = feature_train  
        self.label=label_train  
        self.x = torch.from_numpy(self.feature).float()
        self.y = torch.from_numpy(self.label).float()
        # print(f'feature:{self.x}')
        # print(f'label:{self.y}')
    def __len__(self):
        return len(self.y)
    def __getitem__(self, index):
        x=self.x[index]
        y=self.y[index]
        return {'x':x,'y':y}

dataset=ExcelDataset()
dataloader=DataLoader(dataset=dataset, batch_size=256,shuffle=True,num_workers=0,drop_last = True)

In [ ]:
import torch.nn.init as init
class BP(nn.Module):
    def __init__(self, num_layers,hidden_size):
        super(BP,self).__init__()
        self.Liner1 = nn.Linear(10,hidden_size)
        self.queue= [nn.Sequential(nn.Linear(hidden_size, hidden_size), nn.LeakyReLU(0.1)) for _ in range(num_layers - 1)]
        self.model=nn.Sequential(*self.queue)
        self.Liner2 = nn.Linear(hidden_size,1)
        
        
    def forward(self,input):
        output = self.Liner1(input)
        output=self.model(output)
        output = self.Liner2(output)
        #output=output.squeeze(1)
        return output
n_epochs =50
bp=BP(7,12)
optimizer = torch.optim.Adam(bp.parameters(), lr=0.01)
MSEloss=nn.MSELoss()

In [ ]:
def train(): 
    loss_mean_best = 10000
    loss_train = np.zeros(n_epochs)
    loss_test = np.zeros(n_epochs)
    epoch_best = 1
    for epoch in range(0, n_epochs):
        bp.train()
        accuracy_mean = 0
        loss_mean = 0
        for i,value in enumerate(dataloader):
            x=value['x']
            y=value['y']
            y_train=bp(x)
            y_train=y_train.squeeze(1)
            loss = MSEloss(y,y_train)
            loss_mean += loss.item()
            accuracy = r2_score(y.detach().numpy(),y_train.detach().numpy())
            accuracy_mean += accuracy
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(
            "\r[Epoch %d/%d] [Batch %d/%d] [loss_train: %f]"
            % (
                epoch+1,
                n_epochs,
                i+1,
                len(dataloader),
                loss_mean/len(dataloader),
            ))
        loss_train[epoch] = loss_mean/len(dataloader)
        batches_done = epoch * len(dataloader) + i
        with torch.no_grad():   ## early stop
            _, loss_val1 = test(feature_test,label_test)
            loss_test[epoch] = loss_val1
            if epoch > 20:
                if loss_test[epoch] < loss_mean_best:
                    loss_mean_best = loss_test[epoch]
                    epoch_best = epoch+1
                    torch.save(bp.state_dict(), "bp2_%d.pth" % epoch_best) ##saving best model parameters.
    
    return epoch_best, loss_train, loss_test

In [ ]:
def test(feature_test, label_test):
    predict_list=[]
    bp.eval()
    for test_feature in feature_test:
 
        test_feature = torch.from_numpy(test_feature).float()
        test_feature=test_feature.unsqueeze(0)

        predict=bp(test_feature)
        predict=predict.squeeze(1)

        predict=predict.squeeze(0)
        predict_list.append(predict.detach().numpy())
 
    predict_list=np.array(predict_list)
    loss = MSEloss(torch.from_numpy(predict_list),torch.from_numpy(label_test)).item()
    print("[R2:{}][loss:{}]".format(r2_score(predict_list, label_test), loss))
    return predict_list, loss

In [ ]:
epoch_best,loss_train, loss_test = train()